# Batch SQL Silver Sales

filter(**Bronze_table**) + **Quarentine_table** => **TEMP_Silver**

**TEMP_Silver** => **Silver_table** OR **Quarentine_table**


In [0]:
USE IDENTIFIER(:catalog ||'.'|| :schema)

## Sales


In [0]:
-- Upsert Silver_Sales:
MERGE INTO silver_sales AS T
USING (
  WITH bronze_plus_quarentine AS (
    SELECT 
      Sales_ID,
      Customer_ID,
      Product_ID,
      Revenue,
      Units_Sold,
      Registration_TimeStamp,
      Operation,
      Moviment_Date
    FROM bronze_sales
    -- Filter by day of moviment (moviment_date) :
    WHERE Moviment_Date = :moviment_date

    UNION ALL
    -- Union with sales in quarentine table (wich is expected to be manual updated):
    SELECT
      Sales_ID,
      Customer_ID,
      Product_ID,
      Revenue,
      Units_Sold,
      Registration_TimeStamp,
      Operation,
      Moviment_Date
    FROM quarentine_sales
  ),

  bronze_plus_quarentine_with_rules AS (
    SELECT *, row_number() OVER(PARTITION BY Sales_ID, Registration_TimeStamp ORDER BY Moviment_Date ASC) AS rn,

            -------------------------- Apply rules: 
            CASE WHEN Units_Sold > 0
                THEN 'valid'
                ELSE 'invalid' 
            END AS status
            ---------------------------------------
    FROM bronze_plus_quarentine
  ) 
  SELECT 
    Sales_ID,
    Customer_ID,
    Product_ID,
    Revenue,
    Units_Sold,
    Registration_TimeStamp,
    Operation,
    Moviment_Date
  FROM bronze_plus_quarentine_with_rules
  ------ We just want to keep the valid records to silver table.
  WHERE (Status = 'valid' OR Operation = 'DELETE') AND rn = 1
  --------------------------------------------------------------------------------------------------------------------------
) AS S
  ON T.Sales_ID = S.Sales_ID AND T.Registration_TimeStamp = S.Registration_TimeStamp
-- INSERT NEW RECORD:
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
9,0,0,9


### Quarentine Sales

In [0]:
MERGE INTO quarentine_sales AS T
USING (
  WITH bronze_plus_quarentine AS (
    SELECT
      Sales_ID,
      Customer_ID,
      Product_ID,
      Revenue,
      Units_Sold,
      Registration_TimeStamp,
      Operation,
      Moviment_Date,
      Source_File
    FROM bronze_sales
    -- Filter by day of moviment (moviment_date) and not Sales -2 (artificial Sales genareted to deal with empty file, meaning: no change in operational database for that day of moviment):
    WHERE Moviment_Date = :moviment_date AND Sales_ID <> -2 AND Operation <> 'DELETE'

    UNION ALL
    -- Union with sales in quarentine table (wich is expected to be manual updated):
    SELECT
      Sales_ID,
      Customer_ID,
      Product_ID,
      Revenue,
      Units_Sold,
      Registration_TimeStamp,
      Operation,
      Moviment_Date,
      Source_File
    FROM quarentine_sales
  ),

  bronze_plus_quarentine_with_rules AS (
    SELECT *,

            -------------------------- Apply rules: 
            CASE WHEN Units_Sold > 0
                THEN 'valid'
                ELSE 'invalid' 
            END AS status
            ---------------------------------------
    FROM bronze_plus_quarentine
  ) 
  SELECT *
  FROM bronze_plus_quarentine_with_rules
  ------ We just want to keep the invalid records to quarentine table.
  WHERE Status = 'invalid' 
  --------------------------------------------------------------------------------------------------------------------------
) AS S
  ON T.Sales_ID = S.Sales_ID AND T.Registration_TimeStamp = S.Registration_TimeStamp 
-- UPDATE TO LAST VALID RECORD:
WHEN MATCHED THEN UPDATE SET *
-- INSERT NEW RECORD:
WHEN NOT MATCHED THEN INSERT *
-- Deleted retified registers:
WHEN NOT MATCHED BY SOURCE THEN DELETE

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


In [0]:
UPDATE IDENTIFIER(:catalog ||'.control.control_tables_moviments')
SET MOVIMENT_DATE = :moviment_date
WHERE Project_Name = :project 
  AND Table_name = 'silver_sales'